In [1]:
import pandas as pd
import sqlite3
import csv

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

game_log = pd.read_csv('game_log.csv', low_memory=False)
park_codes = pd.read_csv('park_codes.csv')
person_codes = pd.read_csv('person_codes.csv')
team_codes = pd.read_csv('team_codes.csv')

In [2]:
game_log.shape

(171907, 161)

In [3]:
game_log.head()

date  number_of_game day_of_week v_name v_league  v_game_number h_name  \
0  18710504               0         Thu    CL1      NaN              1    FW1   
1  18710505               0         Fri    BS1      NaN              1    WS3   
2  18710506               0         Sat    CL1      NaN              2    RC1   
3  18710508               0         Mon    CL1      NaN              3    CH1   
4  18710509               0         Tue    BS1      NaN              2    TRO   

  h_league  h_game_number  v_score  h_score  length_outs day_night completion  \
0      NaN              1        0        2         54.0         D        NaN   
1      NaN              1       20       18         54.0         D        NaN   
2      NaN              1       12        4         54.0         D        NaN   
3      NaN              1       12       14         54.0         D        NaN   
4      NaN              1        9        5         54.0         D        NaN   

  forefeit protest park_id  attendance  length_minutes v_line_score  \
0      NaN     NaN   FOR01       200.0           120.0    000000000   
1      NaN     NaN   WAS01      5000.0           145.0    107000435   
2      NaN     NaN   RCK01      1000.0           140.0    610020003   
3      NaN     NaN   CHI01      5000.0           150.0    101403111   
4      NaN     NaN   TRO01      3250.0           145.0    000002232   

  h_line_score  v_at_bats  v_hits  v_doubles  v_triples  v_homeruns  v_rbi  \
0    010010000       30.0     4.0        1.0        0.0         0.0    0.0   
1    640113030       41.0    13.0        1.0        2.0         0.0   13.0   
2    010020100       49.0    11.0        1.0        1.0         0.0    8.0   
3    077000000       46.0    15.0        2.0        1.0         2.0   10.0   
4    101003000       46.0    17.0        4.0        1.0         0.0    6.0   

   v_sacrifice_hits  v_sacrifice_flies  v_hit_by_pitch  v_walks  \
0               0.0                0.0             0.0      1.0   
1               0.0                0.0             0.0     18.0   
2               0.0                0.0             0.0      0.0   
3               0.0                0.0             0.0      0.0   
4               0.0                0.0             0.0      2.0   

   v_intentional_walks  v_strikeouts  v_stolen_bases  v_caught_stealing  \
0                  NaN           6.0             1.0                NaN   
1                  NaN           5.0             3.0                NaN   
2                  NaN           1.0             0.0                NaN   
3                  NaN           1.0             0.0                NaN   
4                  NaN           0.0             1.0                NaN   

   v_grounded_into_double  v_first_catcher_interference  v_left_on_base  \
0                    -1.0                           NaN             4.0   
1                    -1.0                           NaN            12.0   
2                    -1.0                           NaN            10.0   
3                    -1.0                           NaN             7.0   
4                    -1.0                           NaN            12.0   

   v_pitchers_used  v_individual_earned_runs  v_team_earned_runs  \
0              1.0                       1.0                 1.0   
1              1.0                       6.0                 6.0   
2              1.0                       0.0                 0.0   
3              1.0                       6.0                 6.0   
4              1.0                       2.0                 2.0   

   v_wild_pitches  v_balks  v_putouts  v_assists  v_errors  v_passed_balls  \
0             0.0      0.0       27.0        9.0       0.0             3.0   
1             1.0      0.0       27.0       13.0      10.0             1.0   
2             2.0      0.0       27.0       12.0       8.0             5.0   
3             0.0      0.0       27.0       15.0      11.0             6.0   
4             0.0      0.0       

In [4]:
game_log.tail()

date  number_of_game day_of_week v_name v_league  v_game_number  \
171902  20161002               0         Sun    MIL       NL            162   
171903  20161002               0         Sun    NYN       NL            162   
171904  20161002               0         Sun    LAN       NL            162   
171905  20161002               0         Sun    PIT       NL            162   
171906  20161002               0         Sun    MIA       NL            161   

       h_name h_league  h_game_number  v_score  h_score  length_outs  \
171902    COL       NL            162        6        4         60.0   
171903    PHI       NL            162        2        5         51.0   
171904    SFN       NL            162        1        7         51.0   
171905    SLN       NL            162        4       10         51.0   
171906    WAS       NL            162        7       10         51.0   

       day_night completion forefeit protest park_id  attendance  \
171902         D        NaN      NaN     NaN   DEN02     27762.0   
171903         D        NaN      NaN     NaN   PHI13     36935.0   
171904         D        NaN      NaN     NaN   SFO03     41445.0   
171905         D        NaN      NaN     NaN   STL10     44615.0   
171906         D        NaN      NaN     NaN   WAS11     28730.0   

        length_minutes v_line_score h_line_score  v_at_bats  v_hits  \
171902           203.0   0200000202   1100100010       39.0    10.0   
171903           159.0    000001100    00100031x       33.0     8.0   
171904           184.0    000100000    23000002x       30.0     4.0   
171905           192.0    000020200    00100360x       35.0     9.0   
171906           216.0    000230020    03023002x       38.0    14.0   

        v_doubles  v_triples  v_homeruns  v_rbi  v_sacrifice_hits  \
171902        4.0        1.0         2.0    6.0               0.0   
171903        3.0        0.0         0.0    2.0               0.0   
171904        0.0        0.0         0.0    1.0               0.0   
171905        0.0        0.0         1.0    4.0               0.0   
171906        1.0        1.0         2.0    7.0               1.0   

        v_sacrifice_flies  v_hit_by_pitch  v_walks  v_intentional_walks  \
171902                0.0             1.0      4.0                  0.0   
171903                0.0             0.0      2.0                  0.0   
171904                0.0             0.0      2.0                  0.0   
171905                0.0             0.0      4.0                  0.0   
171906                0.0             0.0      3.0                  2.0   

        v_strikeouts  v_stolen_bases  v_caught_stealing  \
171902          12.0             2.0                1.0   
171903           9.0             1.0                1.0   
171904           7.0             0.0                0.0   
171905          11.0             0.0                1.0   
171906          10.0             1.0                1.0   

        v_grounded_into_double  v_first_catcher_interference  v_left_on_base  \
171902                     0.0                           0.0             8.0   
171903                     1.0                           0.0             6.0   
171904                     1.0                           0.0             4.0   
171905                     0.0                           0.0             8.0   
171906                     1.0                           0.0             8.0   

        v_pitchers_used  v_individual_earned_runs  v_team_earned_runs  \
171902              7.0                       4.0                 4.0   
171903              6.0                       3.0                 3.0   
171904              7.0                       7.0                 7.0   
171905              6.0                       8.0                 8.0   
171906              7.0                      10.0                10.0   

        v_wild_pitches  v_balks  v_putouts  v_assists  v_errors  \
171902             1.0      0.0       30.0       12.0       

In [5]:
park_codes.shape

(252, 9)

In [6]:
park_codes.head()

park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                        aka       city state       start  \
0                                       NaN     Albany    NY  09/11/1880   
1                                       NaN    Altoona    PA  04/30/1884   
2             Edison Field; Anaheim Stadium    Anaheim    CA  04/19/1966   
3                                       NaN  Arlington    TX  04/21/1972   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX  04/11/1994   

          end league                                           notes  
0  05/30/1882     NL  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1  05/31/1884     UA                                             NaN  
2         NaN     AL                                             NaN  
3  10/03/1993     AL                                             NaN  
4         NaN     AL                                             NaN

In [7]:
person_codes.shape

(20494, 7)

In [8]:
person_codes.head()

id     last   first player_debut mgr_debut coach_debut ump_debut
0  aardd001  Aardsma   David   04/06/2004       NaN         NaN       NaN
1  aaroh101    Aaron    Hank   04/13/1954       NaN         NaN       NaN
2  aarot101    Aaron  Tommie   04/10/1962       NaN  04/06/1979       NaN
3  aased001     Aase     Don   07/26/1977       NaN         NaN       NaN
4  abada001     Abad    Andy   09/10/2001       NaN         NaN       NaN

In [9]:
team_codes.shape

(150, 8)

In [10]:
team_codes.head()

team_id league  start   end       city         nickname franch_id  seq
0     ALT     UA   1884  1884    Altoona  Mountain Cities       ALT    1
1     ARI     NL   1998     0    Arizona     Diamondbacks       ARI    1
2     BFN     NL   1879  1885    Buffalo           Bisons       BFN    1
3     BFP     PL   1890  1890    Buffalo           Bisons       BFP    1
4     BL1    NaN   1872  1874  Baltimore         Canaries       BL1    1

game_log is a fairly large file consisting of details about each game including the players and the positions they played.

The defensive position number tells us what position each player played on the field. The player's number corresponds with the batting order.

park_codes is a much smaller file telling us about the ballparks.

person_codes has a history of people in the profession with dates of when they debuted in different roles.

team_codes is short and tells us basic information about teams.

In [11]:
game_log['h_player_1_def_pos'].value_counts(dropna=True)

8.0     38373
4.0     28616
6.0     22268
7.0     19560
9.0     15257
5.0     11694
3.0      3444
10.0     1104
2.0       521
1.0         1
Name: h_player_1_def_pos, dtype: int64

Reviewing Wikipedia, we can see that the numerical values of positions correspond to the following:

1. pitcher
2. catcher
3. first baseman
4. second baseman
5. third baseman
6. shortstop
7. left fielder
8. center fielder
9. right fielder
10. (designated hitter)

In [12]:
game_log['h_league'].value_counts(dropna=True)

NL    88867
AL    74712
AA     5039
FL     1243
PL      532
UA      428
Name: h_league, dtype: int64

From knowledge of baseball, I know that the NL and AL are the two current leagues in MLB.

The full list is:

NL: National League
AL: American League
AA: American Association
FL: Federal League
PL: Players' League
UA: Union Association

In [13]:
leagues = ['NL','AL','AA','FL','PL','UA']
date_range = {}
for league in leagues:
    origin_date = game_log[game_log['h_league'] == league]['date'].min()
    latest_date = game_log[game_log['h_league'] == league]['date'].max()
    print("{} has recorded games ranging from {} to {}".format(league, origin_date, latest_date))

NL has recorded games ranging from 18760422 to 20161002
AL has recorded games ranging from 19010424 to 20161002
AA has recorded games ranging from 18820502 to 18911006
FL has recorded games ranging from 19140413 to 19151003
PL has recorded games ranging from 18900419 to 18901004
UA has recorded games ranging from 18840417 to 18841019


Looking at the date ranges, this seems to indicate the last four are no longer around or at least are not part of this dataset's focus.

In [14]:
db = 'mlb.db'

def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)
        
def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)
    
def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

In [15]:
tables = {"game_log": game_log,
         "park_codes": park_codes,
         "person_codes": person_codes,
         "team_codes": team_codes}

with sqlite3.connect(db) as conn:
    for name, data in tables.items():
        conn.execute('DROP TABLE IF EXISTS {};'.format(name))
        data.to_sql(name, conn, index=False)

In [16]:
show_tables()

name   type
0  person_codes  table
1    team_codes  table
2      game_log  table
3    park_codes  table

In [17]:
q1 = '''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
'''

try:
    run_command(q1)
except:
    pass

q2 = '''
UPDATE game_log
SET game_id = h_name || date || number_of_game
WHERE game_id is NULL;
'''

run_command(q2)

In [18]:
test_query = '''
SELECT h_name,
        date,
        number_of_game,
        game_id
FROM game_log
LIMIT 5;
'''

run_query(test_query)

h_name      date  number_of_game       game_id
0    FW1  18710504               0  FW1187105040
1    WS3  18710505               0  WS3187105050
2    RC1  18710506               0  RC1187105060
3    CH1  18710508               0  CH1187105080
4    TRO  18710509               0  TRO187105090

In game_log, we can add new columns for umpire positions, home/away and winning/losing/saving to remove the need for separate columns to indicate details about each. We can also remove columns related to names since those names are given in persons_code with a corresponding id.

In park_codes, we can remove the dates and determine this from the game_log data.

In person_codes, we can remove debut dates as this can be determined in the game_log data.

In team_codes, we can remove start, end and seq columns as they will be determined from game_log data as well.

## Add schema image later if uploaded

## Create new tables

### Create persons table

In [19]:
drop_table = "DROP TABLE IF EXISTS person;"
run_command(drop_table)

create_person_table = '''
CREATE TABLE IF NOT EXISTS person (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);
'''

insert_person_data = '''
INSERT OR IGNORE INTO person
    SELECT
            id,
            first,
            last
    FROM person_codes;
'''

review_person_table = '''
SELECT * FROM person
LIMIT 5;
'''

run_command(create_person_table)
run_command(insert_person_data)
run_query(review_person_table)

person_id first_name last_name
0  aardd001      David   Aardsma
1  aaroh101       Hank     Aaron
2  aarot101     Tommie     Aaron
3  aased001        Don      Aase
4  abada001       Andy      Abad

In [20]:
show_tables()

name   type
0  person_codes  table
1    team_codes  table
2      game_log  table
3    park_codes  table
4        person  table

### Create park table

In [21]:
create_park_table = '''
CREATE TABLE IF NOT EXISTS park (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
);
'''

insert_park_data = '''
INSERT OR IGNORE INTO park
    SELECT
            park_id,
            name,
            aka,
            city,
            state,
            notes
    FROM park_codes;
'''

review_park_table = '''
SELECT * FROM park
LIMIT 5;
'''

run_command(create_park_table)
run_command(insert_park_data)
run_query(review_park_table)

park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                   nickname       city state  \
0                                      None     Albany    NY   
1                                      None    Altoona    PA   
2             Edison Field; Anaheim Stadium    Anaheim    CA   
3                                      None  Arlington    TX   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX   

                                            notes  
0  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1                                            None  
2                                            None  
3                                            None  
4                                            None

### Create league table

In [22]:
create_league_table = '''
CREATE TABLE IF NOT EXISTS league (
    league_id TEXT PRIMARY KEY,
    name TEXT
);
'''

insert_league_data = '''
INSERT OR IGNORE INTO league
VALUES
    ("NL", "National League"),
    ("AL", "American League"),
    ("AA", "American Association"),
    ("FL", "Federal League"),
    ("PL", "Players League"),
    ("UA", "Union Association");
'''

review_league_table = '''
SELECT * FROM league
LIMIT 5;
'''

run_command(create_league_table)
run_command(insert_league_data)
run_query(review_league_table)

league_id                  name
0        NL       National League
1        AL       American League
2        AA  American Association
3        FL        Federal League
4        PL        Players League

### Create appearance_type table

In [23]:
drop = "DROP TABLE IF EXISTS appearance_type"
run_command(drop)

create_app_type_table = '''
CREATE TABLE IF NOT EXISTS appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
);
'''

run_command(create_app_type_table)

appearance_type = pd.read_csv('appearance_type.csv')

with sqlite3.connect(db) as conn:
    appearance_type.to_sql('appearance_type', conn, index=False,
                           if_exists='append')

review_app_type_table = '''
SELECT * FROM appearance_type
LIMIT 5;
'''

run_query(review_app_type_table)

appearance_type_id      name category
0                 O1  Batter 1  offense
1                 O2  Batter 2  offense
2                 O3  Batter 3  offense
3                 O4  Batter 4  offense
4                 O5  Batter 5  offense

### Create team table

In [24]:
create_team_table = '''
CREATE TABLE IF NOT EXISTS team (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) references league(league_id)
);
'''

insert_team_data = '''
INSERT OR IGNORE INTO team
SELECT
        team_id,
        league,
        city,
        nickname,
        franch_id
FROM team_codes;
'''

review_team_table = '''
SELECT * FROM team
LIMIT 5;
'''

run_command(create_team_table)
run_command(insert_team_data)
run_query(review_team_table)

team_id league_id       city         nickname franch_id
0     ALT        UA    Altoona  Mountain Cities       ALT
1     ARI        NL    Arizona     Diamondbacks       ARI
2     BFN        NL    Buffalo           Bisons       BFN
3     BFP        PL    Buffalo           Bisons       BFP
4     BL1      None  Baltimore         Canaries       BL1

In [25]:
show_tables()

name   type
0     person_codes  table
1       team_codes  table
2         game_log  table
3       park_codes  table
4           person  table
5             park  table
6           league  table
7  appearance_type  table
8             team  table

### Create game table

In [26]:
create_game_table = '''
CREATE TABLE IF NOT EXISTS game (
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day BOOLEAN,
    completion TEXT,
    forfeit TEXT,
    protest TEXT,
    attendance INTEGER,
    length_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
);
'''

insert_game_data = '''
INSERT OR IGNORE INTO game
SELECT
        game_id,
        date,
        number_of_game,
        park_id,
        length_outs,
        CASE
            WHEN day_night = 'D' THEN 1
            WHEN day_night = 'N' THEN 0
            ELSE NULL
        END as day,
        completion,
        forefeit,
        protest,
        attendance,
        length_minutes,
        additional_info,
        acquisition_info
FROM game_log;
'''

review_game_table = '''
SELECT * FROM game
LIMIT 5;
'''

run_command(create_game_table)
run_command(insert_game_data)
run_query(review_game_table)

game_id      date  number_of_game park_id  length_outs  day  \
0  FW1187105040  18710504               0   FOR01           54    1   
1  WS3187105050  18710505               0   WAS01           54    1   
2  RC1187105060  18710506               0   RCK01           54    1   
3  CH1187105080  18710508               0   CHI01           54    1   
4  TRO187105090  18710509               0   TRO01           54    1   

  completion forfeit protest  attendance  length_minutes additional_info  \
0       None    None    None         200             120            None   
1       None    None    None        5000             145            HTBF   
2       None    None    None        1000             140            None   
3       None    None    None        5000             150            None   
4       None    None    None        3250             145            HTBF   

  acquisition_info  
0                Y  
1                Y  
2                Y  
3                Y  
4                Y

In [27]:
run_query('''
SELECT * FROM game
WHERE game_id = 'FW1187105040';
''')

game_id      date  number_of_game park_id  length_outs  day  \
0  FW1187105040  18710504               0   FOR01           54    1   

  completion forfeit protest  attendance  length_minutes additional_info  \
0       None    None    None         200             120            None   

  acquisition_info  
0                Y

In [28]:
game_log_cols = run_query('''
SELECT sql FROM sqlite_master
WHERE name = "game_log"
    AND type = "table";
''')

print(game_log_cols)

### Create team_appearance table

In [29]:
create_team_app_table = '''
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
'''

insert_team_app_data = '''
INSERT OR IGNORE INTO team_appearance
    SELECT
            h_name,
            game_id,
            1 AS home,
			h_league,
			h_score,
			h_line_score,
			h_at_bats,
			h_hits,
			h_doubles,
			h_triples,
			h_homeruns,
			h_rbi,
			h_sacrifice_hits,
			h_sacrifice_flies,
			h_hit_by_pitch,
			h_walks,
			h_intentional_walks,
			h_strikeouts,
			h_stolen_bases,
			h_caught_stealing,
			h_grounded_into_double,
			h_first_catcher_interference,
			h_left_on_base,
			h_pitchers_used,
			h_individual_earned_runs,
			h_team_earned_runs,
			h_wild_pitches,
			h_balks,
			h_putouts,
			h_assists,
			h_errors,
			h_passed_balls,
			h_double_plays,
			h_triple_plays
    FROM game_log
    
UNION

    SELECT
            v_name,
			game_id,
			0 AS home,
			v_league,
			v_score,
			v_line_score,
			v_at_bats,
			v_hits,
			v_doubles,
			v_triples,
			v_homeruns,
			v_rbi,
			v_sacrifice_hits,
			v_sacrifice_flies,
			v_hit_by_pitch,
			v_walks,
			v_intentional_walks,
			v_strikeouts,
			v_stolen_bases,
			v_caught_stealing,
			v_grounded_into_double,
			v_first_catcher_interference,
			v_left_on_base,
			v_pitchers_used,
			v_individual_earned_runs,
			v_team_earned_runs,
			v_wild_pitches,
			v_balks,
			v_putouts,
			v_assists,
			v_errors,
			v_passed_balls,
			v_double_plays,
			v_triple_plays
    FROM game_log;
'''

review_team_app_table = '''
SELECT * FROM team_appearance
LIMIT 5;
'''

run_command(create_team_app_table)
run_command(insert_team_app_data)
run_query(review_team_app_table)

team_id       game_id  home league_id  score line_score at_bats  hits  \
0     ALT  ALT188404300     1        UA      2       None    None  None   
1     ALT  ALT188405020     1        UA      3       None    None  None   
2     ALT  ALT188405030     1        UA      5       None    None  None   
3     ALT  ALT188405050     1        UA      2       None    None  None   
4     ALT  ALT188405100     1        UA      9       None    None  None   

  doubles triples homeruns   rbi sacrifice_hits sacrifice_flies hit_by_pitch  \
0    None    None     None  None           None            None         None   
1    None    None     None  None           None            None         None   
2    None    None     None  None           None            None         None   
3    None    None     None  None           None            None         None   
4    None    None     None  None           None            None         None   

  walks intentional_walks strikeouts stolen_bases caught_stealing  \
0  None              None       None         None            None   
1  None              None       None         None            None   
2  None              None       None         None            None   
3  None              None       None         None            None   
4  None              None       None         None            None   

  grounded_into_double first_catcher_interference left_on_base pitchers_used  \
0                 None                       None         None          None   
1                 None                       None         None          None   
2                 None                       None         None          None   
3                 None                       None         None          None   
4                 None                       None         None          None   

  individual_earned_runs team_earned_runs wild_pitches balks putouts assists  \
0                   None             None         None  None    None    None   
1                   None             None         None  None    None    None   
2                   None             None         None  None    None    None   
3                   None             None         None  None    None    None   
4                   None             None         None  None    None    None   

  errors passed_balls double_plays triple_plays  
0   None         None         None         None  
1   None         None         None         None  
2   None         None         None         None  
3   None         None         None         None  
4   None         None         None         None

### Create person_appearance table

In [30]:
game_log.columns.tolist()

['date',
 'number_of_game',
 'day_of_week',
 'v_name',
 'v_league',
 'v_game_number',
 'h_name',
 'h_league',
 'h_game_number',
 'v_score',
 'h_score',
 'length_outs',
 'day_night',
 'completion',
 'forefeit',
 'protest',
 'park_id',
 'attendance',
 'length_minutes',
 'v_line_score',
 'h_line_score',
 'v_at_bats',
 'v_hits',
 'v_doubles',
 'v_triples',
 'v_homeruns',
 'v_rbi',
 'v_sacrifice_hits',
 'v_sacrifice_flies',
 'v_hit_by_pitch',
 'v_walks',
 'v_intentional_walks',
 'v_strikeouts',
 'v_stolen_bases',
 'v_caught_stealing',
 'v_grounded_into_double',
 'v_first_catcher_interference',
 'v_left_on_base',
 'v_pitchers_used',
 'v_individual_earned_runs',
 'v_team_earned_runs',
 'v_wild_pitches',
 'v_balks',
 'v_putouts',
 'v_assists',
 'v_errors',
 'v_passed_balls',
 'v_double_plays',
 'v_triple_plays',
 'h_at_bats',
 'h_hits',
 'h_doubles',
 'h_triples',
 'h_homeruns',
 'h_rbi',
 'h_sacrifice_hits',
 'h_sacrifice_flies',
 'h_hit_by_pitch',
 'h_walks',
 'h_intentional_walks',
 'h_stri

In [32]:
create_person_app_table = '''
CREATE TABLE IF NOT EXISTS person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
'''

insert_person_app_data = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
"""

template = '''
INSERT INTO person_appearance (
    person_id,
    team_id,
    game_id,
    appearance_type_id
) 
    SELECT
        {hv}_player_{num}_id,
        {hv}_name,
        game_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        {hv}_player_{num}_id,
        {hv}_name,
        game_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
'''

run_command(create_person_app_table)
run_command(insert_person_app_data)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        # run commmand is a helper function which runs
        # a query against our database.
        run_command(template.format(**query_vars))

In [33]:
run_query("SELECT * FROM person_appearance LIMIT 5;")

appearance_id person_id team_id       game_id appearance_type_id
0              1  maplb901    None  ALT188404300                UHP
1              2  curte801     ALT  ALT188404300                 MM
2              3  murpj104     ALT  ALT188404300                PSP
3              4  hodnc101     SLU  ALT188404300                PSP
4              5  sullt101     SLU  ALT188404300                 MM

In [34]:
d1 = "DROP TABLE IF EXISTS game_log"
d2 = "DROP TABLE IF EXISTS park_codes"
d3 = "DROP TABLE IF EXISTS team_codes"
d4 = "DROP TABLE IF EXISTS person_codes"

run_command(d1)
run_command(d2)
run_command(d3)
run_command(d4)

In [35]:
show_tables()

name   type
0             person  table
1               park  table
2             league  table
3    appearance_type  table
4               team  table
5               game  table
6    team_appearance  table
7  person_appearance  table